In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pyodbc
import time
%matplotlib inline
from scipy.stats import norm, zscore
import statsmodels
import math as mt

In [80]:
def get_z_score(alpha):
    return norm.ppf(alpha)

# Inputs p-baseline conversion rate which is our estimated p and d-minimum detectable change
# Returns
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    sds=[sd1,sd2]
    return sds

# Inputs:sd1-sd for the baseline,sd2-sd for the expected change,alpha,beta,d-d_min,p-baseline estimate p
# Returns: the minimum sample size required per group according to metric denominator
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

In [81]:
def SQL_Query( UID, password ):

    


    
#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)
    
#Run our sql query

    query = ("""SELECT email_address,customer_id, [2018_may]
    FROM responsys_customer_counts""")

    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [82]:
def cleanCustomers(df, customer_columnname = 'customer_identifier'):
   
    """
    removes rows with blanks, zeroes, alpha-numeric characters and weird longer numbers
    identified as customer numbers. Finally, restores the datatype to 'int64'
    
    Parameters
    ----------
    score_pool : data frame
        Data frame with customer numbers
   
    customer_columnname : string
        Customer number column name from the data frame
        (customerno/customer_identifier/customer_no/customer_id… etc)
   
    Returns
    -------
    dataframe
        A data frame of cleaned customer numbers
    """
   
    df[customer_columnname] = df[customer_columnname].astype(str)
   
    # remove rows with customer number having zeroes and blanks
    shit_long_ints = df[customer_columnname].loc[df[customer_columnname].apply(lambda x: len(str(x)) > 19)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_long_ints)]
    # remove rows with customer number having zeroes and blanks
    shit_alphanums = df[customer_columnname].loc[df[customer_columnname].str.contains('[a-zA-Z]', regex=True)].tolist()
    df = df.loc[~df[customer_columnname].isin(shit_alphanums)]
    # remove rows with customer number having zeroes and blanks
    df = df.loc[~df[customer_columnname].isin(['', '0'])]
   
    df[customer_columnname] = df[customer_columnname].astype('float64')
    df[customer_columnname] = df[customer_columnname].astype('int64')
   
    return df

In [ ]:
temp_DF = SQL_Query( 'Dhananjay.Kumar','password1!')

#temp_DF['2018_may']= temp_DF['2018_may'].astype(int)
#temp_DF['2018_apr']= temp_DF['2018_apr'].astype(int)
#temp_DF['2018_mar']= temp_DF['2018_mar'].astype(int)

#temp_DF = cleanCustomers(temp_DF, customer_columnname = 'customer_id')
#temp_DF.rename(columns={'customer_id': 'customerno'}, inplace=True)

In [ ]:
temp_DF.shape

In [ ]:
temp_DF.columns

In [27]:
temp_DF = pd.read_csv('/Users/sanjaygopinath/Documents/Data/Email_Holdout_Analysis/2018_CLV_Scored.csv', header=0)
#temp_DF.rename(columns={0:'customerno', 1:'email', 2:'channel'}, inplace=True)
#temp_DF = cleanCustomers(temp_DF, customer_columnname='customerno')
del temp_DF['Unnamed: 0']

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
responsys_DF = pd.read_csv('/Users/sanjaygopinath/Documents/Data/Email_Holdout_Analysis/10M_Openers_Master_Filter20190115.txt')

In [28]:
df = pd.read_csv('/Users/sanjaygopinath/Documents/Data/Email_Holdout_Analysis/2018_CLV_Scored.csv', header=0)
del df['Unnamed: 0']
print(df.shape)
#df  = df[df.BG_cond_purch_12 >=0]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(5795794, 14)


In [16]:
temp_DF.LastOrderDate.max()

'2018-12-30'

In [29]:
finalDF = df
del df

In [22]:
responsys_DF.columns


Index(['CUSTOMER_ID_'], dtype='object')

In [30]:
len(set(finalDF.customerno).intersection(set(responsys_DF.CUSTOMER_ID_)))

1948313

In [33]:
finalDF = finalDF[finalDF.customerno.isin(responsys_DF.CUSTOMER_ID_)]

In [35]:
finalDF['weekly_prob'] = finalDF.frequency/finalDF['T']

In [9]:
finalDF = finalDF[finalDF.weekly_prob > .02]

In [ ]:
responsys_customer_counts

In [ ]:
df.columns

In [19]:
finalDF = pd.merge(temp_DF, df, how='inner', on='customerno')

NameError: name 'df' is not defined

In [36]:
finalDF = finalDF[np.isfinite(finalDF.weekly_prob)]

In [ ]:
#optional, we have a lot of customers who are predicted to make no purchases (less than .05). They 
#make up the first deciles, let's reduce that by 2 deciles 



In [37]:
#Cut our customers into deciles based on their predicted spend 
finalDF['pred_spend_bins']= pd.qcut(finalDF['weekly_prob'], 10, labels=[0,1,2,3,4,5,6,7,8,9], retbins=False, precision=3, duplicates='raise')

In [74]:
#Create a summary frame with test group sizes 
# Sample size m

summaryFrame = finalDF[['recency', 'frequency', 'order_count', 'T', 'weekly_prob', 'BG_cond_purch_12', 'BG_pred_spend']].groupby(finalDF.pred_spend_bins).mean()
summaryFrame.reset_index(inplace=True)
summaryFrame['sds'] = summaryFrame.apply(lambda x: (get_sds(x['weekly_prob'], x['weekly_prob']/5)), axis=1)
summaryFrame['sample_size'] = summaryFrame.apply(lambda x: (get_sampSize(sds=x['sds'], alpha=.10, beta=.2, d=x['weekly_prob']/10)), axis=1)
summaryFrame.sample_size = summaryFrame.sample_size.astype(int)

In [73]:
summaryFrame

,pred_spend_bins,recency,frequency,order_count,T,weekly_prob,BG_cond_purch_12,BG_pred_spend,sds,sample_size
0,0,0.000000,1.000000,1.088219,45.259336,0.022286,0.001345,0.058455,"[0.20875326091162452, 0.21866988981551005]",70848
1,1,2.829294,1.150696,1.258670,34.090642,0.033636,0.104407,4.761487,"[0.2549690769030927, 0.2669056424129873]",46377
2,2,8.876611,1.508044,1.640525,32.966244,0.046269,0.437996,20.577342,"[0.29708047381363895, 0.3107550262454191]",33258
3,3,13.424245,1.972693,2.153750,32.409370,0.060923,0.924866,44.338984,"[0.3382648787310144, 0.35351753602423175]",24856
4,4,17.197567,2.527286,2.766388,32.270414,0.078107,1.619164,78.214680,"[0.3794899114519059, 0.39616875616335384]",19020
5,5,19.091677,3.101840,3.411355,30.815126,0.100793,2.363780,114.863807,"[0.42575451823019855, 0.44379658066808847]",14363
6,6,24.426597,4.394784,4.870978,33.618018,0.131283,3.778857,186.363116,"[0.47759301237749396, 0.4967576466569687]",10639
7,7,25.540651,5.652329,6.333568,32.295714,0.177086,5.253392,261.327270,"[0.5398645585155446, 0.5595290921461552]",7455
8,8,32.409519,9.448310,10.891011,37.142626,0.253745,8.950069,451.770673,"[0.6153998116105553, 0.6333528697616676]",4697
9,9,24.985571,13.388699,20.482363,27.403468,0.530849,13.629160,690.484867,"[0.7057596891488465, 0.6930183106550859]",1372


In [40]:
finalDF[['recency', 'frequency', 'order_count', 'T', 'BG_cond_purch_12']].groupby(finalDF.pred_spend_bins).mean()

,recency,frequency,order_count,T,BG_cond_purch_12
pred_spend_bins,,,,,
0,0.000000,1.000000,1.088219,45.259336,0.001345
1,2.829294,1.150696,1.258670,34.090642,0.104407
2,8.876611,1.508044,1.640525,32.966244,0.437996
3,13.424245,1.972693,2.153750,32.409370,0.924866
4,17.197567,2.527286,2.766388,32.270414,1.619164
5,19.091677,3.101840,3.411355,30.815126,2.363780
6,24.426597,4.394784,4.870978,33.618018,3.778857
7,25.540651,5.652329,6.333568,32.295714,5.253392
8,32.409519,9.448310,10.891011,37.142626,8.950069


In [ ]:
summaryFrame['sample_size'].sum()

In [ ]:
#df['BG_pred_spend']= np.round(df['BG_pred_spend'], 0)

In [ ]:
df['BG_pred_spend'].groupby(df['pred_spend_bins']).mean()

In [ ]:
# sort values on two columns 
df.sort_values(['BG_cond_purch_12', 'BG_pred_spend'], ascending=[False, False], inplace=True)

In [57]:
finalDF.pred_spend_bins.value_counts()

2    199847
0    196953
1    196455
7    194989
5    194727
6    194509
4    194202
9    194055
8    193808
3    185815
Name: pred_spend_bins, dtype: int64

In [54]:
summaryFrame

,pred_spend_bins,recency,frequency,order_count,T,weekly_prob,BG_cond_purch_12,BG_pred_spend,sds,sample_size
0,0,0.000000,1.000000,1.088219,45.259336,0.022286,0.001345,0.058455,"[0.20875326091162452, 0.21378070261576876]",69869
1,1,2.829294,1.150696,1.258670,34.090642,0.033636,0.104407,4.761487,"[0.2549690769030927, 0.26102727801156644]",45745
2,2,8.876611,1.508044,1.640525,32.966244,0.046269,0.437996,20.577342,"[0.29708047381363895, 0.3040298596543196]",32813
3,3,13.424245,1.972693,2.153750,32.409370,0.060923,0.924866,44.338984,"[0.3382648787310144, 0.3460289071642701]",24531
4,4,17.197567,2.527286,2.766388,32.270414,0.078107,1.619164,78.214680,"[0.3794899114519059, 0.38799760973644465]",18778
5,5,19.091677,3.101840,3.411355,30.815126,0.100793,2.363780,114.863807,"[0.42575451823019855, 0.4349859180399537]",14187
6,6,24.426597,4.394784,4.870978,33.618018,0.131283,3.778857,186.363116,"[0.47759301237749396, 0.48744638056895406]",10516
7,7,25.540651,5.652329,6.333568,32.295714,0.177086,5.253392,261.327270,"[0.5398645585155446, 0.5500698769522535]",7377
8,8,32.409519,9.448310,10.891011,37.142626,0.253745,8.950069,451.770673,"[0.6153998116105553, 0.6249562048821155]",4659
9,9,24.985571,13.388699,20.482363,27.403468,0.530849,13.629160,690.484867,"[0.7057596891488465, 0.701429654544187]",1382


In [63]:
#Let's create a sampling procedure 

#N is the number of independent trials we will be running ()
n=4

#Create a temporary df to sample without replacement from 
samplingDF = finalDF[['customerno', 'pred_spend_bins']]
sampledFrame = pd.DataFrame()

#Size of sample for each cohort 

print(samplingDF.shape)
#Create a dictionary with cohort name and sample size 
sampleSizeDict = dict(zip(summaryFrame.pred_spend_bins, summaryFrame.sample_size.values)) 
print(sampleSizeDict)
#Create a function that iterates through the dictionary and creates a new dataframe with the samples
for key in sampleSizeDict: 
    tempSamplingDF = samplingDF[samplingDF.pred_spend_bins == key]
    
    #Since we need multiple test groups, why not sample them all at once and then we can assign them to different
    #test periods later. IF yo look below, you can see that we multiply the sample size by n, meaning we have 
    #n different test groups
    tempSampledDF = tempSamplingDF.sample(n*sampleSizeDict.get(key), replace=False)
    
    #We're now randomly assigning these groups to the n different test cohorts
    tempSampledDF.reset_index(inplace=True)
    tempSampledDF.reset_index(inplace=True)
    del tempSampledDF['index']
    tempSampledDF.rename(columns={'level_0':'groups'}, inplace=True)
    
    tempSampledDF['groups'] = tempSampledDF['groups'].apply(lambda x: x%n)
    sampledFrame = sampledFrame.append(tempSampledDF, ignore_index=True)
    
    





(1945360, 2)
{0: 47864, 1: 31338, 2: 22479, 3: 16805, 4: 12864, 5: 9719, 6: 7204, 7: 5054, 8: 3192, 9: 947}


In [65]:
sampledFrame.groups.value_counts()

3    157466
2    157466
1    157466
0    157466
Name: groups, dtype: int64

In [78]:
sampledFrame.to_csv('final_email_test_control_groups.csv')

In [41]:
## Let's create a list of customers to sample based on the necessary size of each cohort 

#First let's create a list of customers based on weekly purchase probability decile 

for i in finalDF.pred_spend_bins.unique():
    list_name = str(i)+'_DecileList'
    list_name = str(i)+'_DecileList'
    #np.random.choice(a, size=None, replace=True, p=None)
    
    print(list_name)

8_DecileList
9_DecileList
6_DecileList
7_DecileList
4_DecileList
3_DecileList
5_DecileList
1_DecileList
2_DecileList
0_DecileList


In [ ]:
#Our goal is to create two datasets of roughly equal shape and equal values 
#We will do this by sorting our dataframe along two columns and then taking every other row 
#and then repeating to take everything else 

# sort values on two columns 
df.sort_values(['BG_cond_purch_12', 'BG_pred_spend'], ascending=[False, False], inplace=True)

#Take every other row in our total dataframe and make a

df2 = df.iloc[::2, :]

#Take all the remaining rows 
#df3 = df[~df.isin(df2)]
#df3.dropna(inplace=True)
df3 = df.iloc[1::2, :]

#df.shape[0] == df2.shape[0] + df3.shape[0]

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].hist(df.BG_cond_purch_12, normed=True, color='blue')

hist, bins = np.histogram(df.BG_cond_purch_12)
ax[1].bar(bins[:-1], hist.astype(np.float32) / hist.sum(), width=(bins[1]-bins[0]), color='red')

ax[0].set_title('normed=True')
ax[1].set_title('hist = hist / hist.sum()')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].hist(df3.BG_cond_purch_12, normed=True, color='grey')

hist, bins = np.histogram(df3.BG_cond_purch_12)
ax[1].bar(bins[:-1], hist.astype(np.float32) / hist.sum(), width=(bins[1]-bins[0]), color='grey')

ax[0].set_title('normed=True')
ax[1].set_title('hist = hist / hist.sum()')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].hist(df2.BG_cond_purch_12, normed=True, color='grey')

hist, bins = np.histogram(df2.BG_cond_purch_12)
ax[1].bar(bins[:-1], hist.astype(np.float32) / hist.sum(), width=(bins[1]-bins[0]), color='grey')

ax[0].set_title('normed=True')
ax[1].set_title('hist = hist / hist.sum()')

In [ ]:
np.round(df2.describe(),1) 

In [ ]:
np.round(df3.describe(),1) 

In [ ]:
#Try to find matching vectors 

np.where((vals == (0, 1)).all(axis=1))
(array([ 3, 15]),)




In [ ]:
from bisect import bisect_left

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before